# PyCaret Binary Classification Tutorial
## Credit Card Fraud Detection Dataset

### Objective
This notebook demonstrates binary classification using PyCaret on a credit card fraud detection dataset.
We will predict whether a transaction is fraudulent or legitimate.

### Dataset
Credit Card Fraud Detection from Kaggle (synthetic version for demonstration)

### Key Features
- AutoML with PyCaret
- GPU acceleration enabled
- Model comparison and selection
- Hyperparameter tuning
- Model interpretation
- Deployment-ready model

## Step 1: Install PyCaret

In [ ]:
!pip install pycaret[full] -q
print("PyCaret installed successfully!")

## Step 2: Import Libraries

In [ ]:
from pycaret.classification import *
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Step 3: Load and Prepare Dataset

In [ ]:
# Generate synthetic credit card fraud data
# In real scenario, use: data = pd.read_csv('creditcard.csv')

X, y = make_classification(
    n_samples=5000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    weights=[0.95, 0.05],  # Imbalanced dataset (95% legitimate, 5% fraud)
    flip_y=0.01,
    random_state=42
)

# Create DataFrame
feature_names = [f'V{i}' for i in range(1, 21)]
data = pd.DataFrame(X, columns=feature_names)
data['Amount'] = np.random.uniform(1, 1000, 5000)
data['Class'] = y  # 0 = Legitimate, 1 = Fraud

print("Dataset Shape:", data.shape)
print("\nFirst 5 rows:")
print(data.head())

print("\nClass Distribution:")
print(data['Class'].value_counts())
print("\nClass Distribution (%)")
print(data['Class'].value_counts(normalize=True) * 100)

## Step 4: Initialize PyCaret Setup with GPU

In [ ]:
# Setup PyCaret environment with GPU acceleration
clf_setup = setup(
    data=data,
    target='Class',
    session_id=42,
    use_gpu=True,  # Enable GPU acceleration
    fix_imbalance=True,  # Handle imbalanced dataset
    normalize=True,
    transformation=True,
    train_size=0.8,
    fold=5,
    verbose=True
)

## Step 5: Compare All Models

In [ ]:
# Compare all available models
best_models = compare_models(
    n_select=5,  # Select top 5 models
    sort='AUC',  # Sort by AUC score
    turbo=True   # Fast mode
)

print("\nTop 5 models selected!")

## Step 6: Create and Train Best Model

In [ ]:
# Create the best model
best = create_model('lightgbm', fold=10)

print("\nLightGBM model created!")

## Step 7: Tune Hyperparameters

In [ ]:
# Tune hyperparameters
tuned_model = tune_model(
    best,
    optimize='AUC',
    n_iter=50,  # Number of iterations
    search_library='optuna',
    search_algorithm='tpe'
)

print("\nModel tuned successfully!")

## Step 8: Ensemble Model

In [ ]:
# Create ensemble model for better performance
bagged_model = ensemble_model(
    tuned_model,
    method='Bagging',
    n_estimators=10
)

print("\nEnsemble model created!")

## Step 9: Model Evaluation

In [ ]:
# Plot AUC curve
plot_model(bagged_model, plot='auc')

In [ ]:
# Plot confusion matrix
plot_model(bagged_model, plot='confusion_matrix')

In [ ]:
# Plot feature importance
plot_model(bagged_model, plot='feature')

In [ ]:
# Plot precision-recall curve
plot_model(bagged_model, plot='pr')

In [ ]:
# Plot class report
plot_model(bagged_model, plot='class_report')

## Step 10: Model Interpretation with SHAP

In [ ]:
# SHAP summary plot
interpret_model(bagged_model)

In [ ]:
# SHAP correlation plot
interpret_model(bagged_model, plot='correlation')

## Step 11: Make Predictions

In [ ]:
# Make predictions on test set
holdout_predictions = predict_model(bagged_model)

print("\nPredictions on holdout set:")
print(holdout_predictions.head(10))

## Step 12: Finalize and Save Model

In [ ]:
# Finalize model (train on entire dataset)
final_model = finalize_model(bagged_model)

print("\nModel finalized!")

In [ ]:
# Save model
save_model(final_model, 'fraud_detection_model')

print("\nModel saved as 'fraud_detection_model.pkl'")

## Step 13: Load and Test Saved Model

In [ ]:
# Load saved model
loaded_model = load_model('fraud_detection_model')

# Test on new data
new_data = data.sample(5).drop('Class', axis=1)
predictions = predict_model(loaded_model, data=new_data)

print("\nPredictions on new data:")
print(predictions)

## Step 14: Model Performance Summary

In [ ]:
# Get model performance metrics
from pycaret.classification import pull

metrics = pull()
print("\nModel Performance Metrics:")
print(metrics)

## Summary

### What We Accomplished:
1. Loaded and prepared imbalanced credit card fraud dataset
2. Initialized PyCaret with GPU acceleration
3. Compared multiple ML models automatically
4. Selected and trained the best model (LightGBM)
5. Performed hyperparameter tuning with Optuna
6. Created ensemble model for improved performance
7. Evaluated model with multiple visualizations
8. Interpreted model with SHAP values
9. Made predictions on new data
10. Saved and loaded the final model

### Key Takeaways:
- PyCaret automates the entire ML pipeline
- GPU acceleration speeds up training significantly
- Handling imbalanced datasets is crucial for fraud detection
- Model interpretation helps understand predictions
- The final model is production-ready and can be deployed